## Predicting the target locations to open a Chinese Restaurant in Pune, India

### Introduction: Business Problem

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening a Chinese restaurant in Pune, India.
Since there are lots of restaurants in Pune we will try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no Chinese restaurants in vicinity. We would also prefer locations as close to city centre as possible, assuming that first two conditions are met.
We will use our data science powers to generate a few most promising neighbourhoods based on this criterion. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

### Data

Based on definition of our problem, factors that will influence our decision are:
* number of existing restaurants in the neighbourhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighbourhood, if any
* distance of neighbourhood from city centre





We decided to use regularly spaced grid of locations, centred around city centre, to define our neighbourhoods.
Following data sources will be needed to extract/generate the required information:
* centres of candidate areas will be generated algorithmically and approximate addresses of centres of those areas will be obtained using Geopy Nominatim
* number of restaurants and their type and location in every neighbourhood will be obtained using Foursquare API
* coordinate of Pune centre will be obtained using Geopy Nominatim of well-known Pune location (Shivajinagar)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Pune city center.

Let's first find the latitude & longitude of Pune city center, using specific, well known address and Geopy Nominatim.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [3]:
def get_coordinates(address, verbose=False):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return [latitude, longitude]

address = 'Shivajinagar, Pune, IN'
pune_center = get_coordinates(address)
print('Coordinate of {}: {}'.format(address, pune_center))

Coordinate of Shivajinagar, Pune, IN: [18.5322493, 73.8499601124847]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Shivajinagar. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [4]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Pune center longitude={}, latitude={}'.format(pune_center[1], pune_center[0]))
x, y = lonlat_to_xy(pune_center[1], pune_center[0])
print('Pune center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Pune center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Pune center longitude=73.8499601124847, latitude=18.5322493
Pune center UTM X=7720707.70280034, Y=3664141.8725536014
Pune center longitude=73.84996011248452, latitude=18.5322492999953


Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is **equally distant from all it's neighbors.**

In [5]:
pune_center_x, pune_center_y = lonlat_to_xy(pune_center[1], pune_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = pune_center_x - 6000
x_step = 600
y_min = pune_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(pune_center_x, pune_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [6]:
!pip install folium

import folium

     |████████████████████████████████| 94 kB 5.2 MB/s  eta 0:00:01


In [11]:
map_pune = folium.Map(location=pune_center, zoom_start=13)
folium.Marker(pune_center, popup='Shivajinagar').add_to(map_pune)
for lat, lon in zip(latitudes, longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='red', fill=True, fill_color='blue', fill_opacity=1).add_to(map_pune) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_pune)
    #folium.Marker([lat, lon]).add_to(map_pune)
map_pune

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Shivajinagar.

Let's now use Geopy Nominatim to get approximate addresses of those locations.

In [16]:
print(pune_center[0], pune_center[1])

18.5322493 73.8499601124847


In [17]:
def get_address(latitude, longitude, verbose=False):
    geolocator = Nominatim(user_agent="foursquare_agent")
    address = geolocator.reverse(str(latitude) + "," + str(longitude))
    return address

addr = get_address(pune_center[0], pune_center[1])
print('Reverse geopy check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(pune_center[0], pune_center[1], addr))

Reverse geopy check
-----------------------
Address of [18.5322493, 73.8499601124847] is: Shivajinagar, Shivajinagar Road, Shivajinagar, Pune City, Pune District, Maharashtra, 411016, India


In [20]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = str(address).replace(', India', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [21]:
addresses[15:50]

['Madhavrao Peshve Marg, Dattawadi, Pune City, Pune District, Maharashtra, 411030',
 'Dattawadi, Pune City, Pune District, Maharashtra, 411030',
 'ARAI Road, Automotive Research Association of India (ARAI), Erandwana, Pune City, Pune District, Maharashtra, 411 004',
 'ARAI Road, Automotive Research Association of India (ARAI), Erandwana, Pune City, Pune District, Maharashtra, 411 004',
 'ILS Law College, Prabhat Path, Erandwana, Pune City, Pune District, Maharashtra, 411004',
 'Jalsa, Krushna Path, Erandwana, Pune City, Pune District, Maharashtra, 411004',
 'Kumudini Pednekar Path, Erandwana, Pune City, Pune District, Maharashtra, 411004',
 'Nanasaheb Bhonde Path, Erandwana, Pune City, Pune District, Maharashtra, 411001',
 'Pune Central, Kashibai Amrutrao Khilare Path, Erandwana, Pune City, Pune District, Maharashtra, 411004',
 'Nivara, Sant Tukaram Maharaj Road, Navi Peth, Pune City, Pune District, Maharashtra, 411030',
 'Navi Peth, Pune City, Pune District, Maharashtra, 411001',
 'Ba

Looking good. Let's now place all this into a Pandas dataframe.

In [22]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"SNDT Women's University, Karve Road, Erandwana...",18.510122,73.826379,7.718908e+06,3.658426e+06,5992.495307
1,"Babasaheb Modak Path, Erandwana, Pune City, Pu...",18.508640,73.829304,7.719508e+06,3.658426e+06,5840.376700
2,"Saint Crispin's Church, Amrutrao Khilare Path,...",18.507158,73.832229,7.720108e+06,3.658426e+06,5747.173218
3,"Erandwana, Pune City, Pune District, Maharasht...",18.505677,73.835154,7.720708e+06,3.658426e+06,5715.767665
4,"Dattawadi, Pune City, Pune District, Maharasht...",18.504195,73.838078,7.721308e+06,3.658426e+06,5747.173218
5,"Dattawadi, Pune City, Pune District, Maharasht...",18.502714,73.841002,7.721908e+06,3.658426e+06,5840.376700
6,"Dattawadi, Pune City, Pune District, Maharasht...",18.501232,73.843926,7.722508e+06,3.658426e+06,5992.495307
7,"ARAI Road, Automotive Research Association of ...",18.514761,73.823336,7.718008e+06,3.658946e+06,5855.766389
8,"Film and Television Institute of India, Vishnu...",18.513279,73.826262,7.718608e+06,3.658946e+06,5604.462508
9,"Café Coffee Day, Vishnushastri Chiplunkar Path...",18.511797,73.829187,7.719208e+06,3.658946e+06,5408.326913


saving this data into local file.

In [23]:
df_locations.to_pickle('./locations.pkl')   

## Foursquare

Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Chinese restaurant' category, as we need info on Chinese restaurants in the neighborhood.


In [27]:
# @hidden_cell
foursquare_client_id = '2TC3VJON2F3DQB0ZQLFSEP0L53HWDBHL1PVLFJWKNRFSHJX1' # your Foursquare ID
foursquare_client_secret = 'WQOAST3QFXCFF2C5YESLTAVKHVRLOTLEKANY3LADHQPFJUDI' # your Foursquare Secret

In [26]:
# Category IDs corresponding to Chinese restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

chinese_restaurant_categories = ['4bf58dd8d48988d145941735','52af3a5e3cf9994f4e043bea','52af3a723cf9994f4e043bec','52af3a7c3cf9994f4e043bed',
                                 '58daa1558bbb0b01f18ec1d3','52af3a673cf9994f4e043beb','52af3a903cf9994f4e043bee','4bf58dd8d48988d1f5931735',
                                 '52af3a9f3cf9994f4e043bef','52af3aaa3cf9994f4e043bf0','52af3ab53cf9994f4e043bf1','52af3abe3cf9994f4e043bf2',
                                 '52af3ac83cf9994f4e043bf3','52af3ad23cf9994f4e043bf4','52af3add3cf9994f4e043bf5','52af3af23cf9994f4e043bf7',
                                 '52af3ae63cf9994f4e043bf6','52af3afc3cf9994f4e043bf8','52af3b053cf9994f4e043bf9','52af3b213cf9994f4e043bfa',
                                 '52af3b293cf9994f4e043bfb','52af3b343cf9994f4e043bfc','52af3b3b3cf9994f4e043bfd','52af3b463cf9994f4e043bfe',
                                 '52af3b633cf9994f4e043c01','52af3b513cf9994f4e043bff','52af3b593cf9994f4e043c00','52af3b6e3cf9994f4e043c02',
                                 '52af3b773cf9994f4e043c03','52af3b813cf9994f4e043c04','52af3b893cf9994f4e043c05','52af3b913cf9994f4e043c06',
                                 '52af3b9a3cf9994f4e043c07','52af3ba23cf9994f4e043c08']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', India', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20210324'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [28]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found chinese restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_chinese = is_restaurant(venue_categories, specific_filter=chinese_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_chinese:
                    chinese_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, chinese_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
chinese_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('chinese_restaurants_350.pkl', 'rb') as f:
        chinese_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, chinese_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('chinese_restaurants_350.pkl', 'wb') as f:
        pickle.dump(chinese_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.
